# Lab02 - Point-based Processing

### CDS6334 Visual Information Processing


This lab introduces the concept of *point-based* processing. As before, this lab is primarily guided in many portions and there are sections with questions that you are attempt to complete. 

First, load the necessary libraries...

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

## I. Image Transformation

Point-based processing involves the direct transformation of each pixel value in an image. An image $f(x,y)$ is said to have undergone a transformation $T$ to an output image $g(x,y)$. In other words, each pixel in $f$ is put thru a function $T$ to get a new value $g$:

\begin{equation}g(x,y)= T(f(x,y))\end{equation}

### A. Negative Image
One of the simplest functions that we can apply to an image is the <b>negative function</b>. 

\begin{equation}g=(L-1)-f\end{equation}

So each value is subtracted by 255. So what happens is that the lighter pixels become dark and the darker picture becomes light. And it results in image negative.

We shall use the image of a pouting girl:
<img src="pout.png" style="width:150px">

In [ ]:
pout = cv2.imread('pout.png',0) 
pout.ndim

In [ ]:
negpout = 255-pout   # 256-1-pout
plt.imshow(pout, cmap='gray')
plt.imshow(np.hstack((pout,negpout)), cmap='gray')
plt.show()

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)#np.nan)   # this is to print full numpy array
print(negpout)

### B. Log Transformation

Non-linear functions can also be applied. They offer a lot more uses especially when the condition of the original image is poor to begin with. The <b>log transformation function</b> is one which is able to stretch the distribution of the pixels:

\begin{equation}
g=c. \text{log} (1+f)
\end{equation}

Where g and f are the pixel values of the output and the input image and c is a constant. The value 1 is added to each of the pixel value of the input image because if there is a pixel intensity of 0 in the image, then log(0) is equal to infinity. So 1 is added, to make the minimum value at least 1.

During log transformation, the dark pixels in an image are expanded as compared to the higher pixel values. The higher pixel values are kind of compressed in log transformation. This result in the following image enhancement.

The value of c in the log transform adjust the kind of enhancement we are looking for.

In [ ]:
c = 25
logpout = np.uint8(c*np.log(1.000001+pout))
logpout
print(logpout.dtype)
plt.subplot(121), plt.imshow(pout, cmap='gray')
plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(logpout, cmap='gray')
plt.xticks([]), plt.yticks([])
plt.show()

### C. Gamma Correction

<b>Gamma correction</b> is a nonlinear operation used to encode and decode luminance or tristimulus values in video or still image systems. Gamma correction is also known as the <b>Power Law Transform</b>. First, the image pixel intensities must be scaled from the range 0, 255 to 0, 1.0. From there, we obtain our output gamma corrected image by applying the following equation:
\begin{equation} g = c.f^{(1/\gamma)} \end{equation}

where $f$ is the input image and $\gamma$ is the gamma value. The output image, $f$ is then scaled back to the range 0-255.

<b>The Reason of Gamma Correction</b>

The reason we apply gamma correction is that our eyes perceive color and luminance differently than the sensors in a digital camera. When a sensor on a digital camera picks up twice the amount of photons, the signal is doubled. However, our eyes do not work like this. Instead, our eyes perceive double the amount of light as only a fraction brighter. Thus, while a digital camera has a linear relationship between brightness our eyes have a non-linear relationship. In order to account for this relationship, we apply gamma correction.


In [ ]:
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    # print(table)
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

pout
plt.figure(figsize=(10,4))
plt.subplot(131), plt.imshow(adjust_gamma(pout, 1), cmap='gray')
plt.subplot(132), plt.imshow(adjust_gamma(pout, 0.5), cmap='gray')
plt.subplot(133), plt.imshow(adjust_gamma(pout, 5),cmap='gray')
plt.tight_layout()
plt.show()

**Q1**: Apply the (1) <b>negative</b>, (2) <b>log transformation</b> and (3) <b>gamma</b> function to the following color image, "parrot.png". For log transformation, apply to the gray image of the parrot.<br>
<img src="parrot.png" style="width:250px">

In [ ]:
#Enter code here


**Q2**: Try to implement the following three functions on the `pout.png` image:<br>
<img src="arithmetic-operations.png" style="width:500px">

In [ ]:
#Enter code here


## II. Histograms

An image histogram is typically a graphical plot that shows the frequency of occurrence (or distribution) of intensity levels in an image *(Note: Later, we will learn that histograms are simply a vector of count values that can be used for more higher-level tasks like representation and description.)* Two concepts are most important in a histogram:
- BINS: the number of compartments we wish to use to group the intensity values into. Typically, we just use a bin for every intensity value (hence, 256 bins in total), or we can also use larger bins that can take a range of values, i.e. 16 bins for each 16 intensity values.
- RANGE: the range of intensity values that you want to measure. Normally it's [0, 256]

There are two functions that can be used to calculate histograms (one from OpenCV, the other from Numpy). Check out the documentation to find out more about the input parameters for [`cv2.calcHist`](https://docs.opencv.org/3.4.1/d6/dc7/group__imgproc__hist.html) and [`np.histogram`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html).

In [ ]:
pout = cv2.imread('pout.png', cv2.IMREAD_GRAYSCALE)
hist0 = cv2.calcHist([pout],[0],None,[256],[0,256])    # using OpenCV function
hist,bins = np.histogram(pout.ravel(),256,[0,256])     # using Numpy function

*Note: OpenCV function is faster (around 40x) than np.histogram(). So take this point into consideration when efficiency is a priority.*

There are two ways to plot the histogram -- using Matplotlib and using OpenCV drawing functions. We will stick to using Matplotlib as it is more straightforward.

In [ ]:
# Using matplotlib
plt.figure(figsize=(8,8))
plt.subplot(121), plt.imshow(pout, cmap='gray')
plt.subplot(122), plt.hist(pout.ravel(),256,[0,256]); 
plt.show()

In [ ]:
def showImage(img, titlestr="" ):
    if img.ndim == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img)
    plt.xticks([]), plt.yticks([])  
    plt.title(titlestr)
    plt.show()
    
# Using OpenCV
rf = cv2.imread('redflower.jpg')
rf = cv2.cvtColor(rf, cv2.COLOR_BGR2RGB)  
showImage(rf)
plt.figure(figsize=(7,3))
color = ('r','g','b')
for i,col in enumerate(color):
    histr = cv2.calcHist([rf],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()

Noticed how the individual red, green and blue channels are distributed across the intensity values?
Tinker with the number of bins to make more observations.

#### Application of Mask

We used <b>cv2.calcHist()</b> to find the histogram of the full image. What if you want to find histograms of some regions of an image? Just create a mask image with white color on the region you want to find histogram and black otherwise. Then pass this as the mask.

The mask is created in the following code snipplet. Complete the program to create and show the histogram with the mask.


In [ ]:
# create a mask
mask = np.zeros(rf.shape[:2], np.uint8)
mask[50:150, 50:150] = 255
masked_img = cv2.bitwise_and(rf,rf,mask = mask)

# create and show the histogram with the mask
showImage(masked_img)
plt.figure(figsize=(7,3))
color = ('r','g','b')
for i,col in enumerate(color):
    histr = cv2.calcHist(???) #enter your code here
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()




### Histogram Equalization

Histogram equalization attemps to improve the distribution of intensity values in an image by equalizing or "flattening" the histogram as much as possible. 

In [ ]:
hist,bins = np.histogram(pout.flatten(),256,[0,256])

# this finds the cdf of the histogram 
cdf = hist.cumsum()      

cdf_normalized = cdf * hist.max()/ cdf.max()     # normalize just to scale values down to show it clearly within histogram
plt.plot(cdf_normalized, color = 'b')
plt.hist(pout.flatten(),256,[0,256], color = 'r')
plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')
plt.show()

You can see that the histogram lies a narrow middle range. We need it to occupy the full spectrum. In pixel- or point-based image transformation, we need a transformation function which spreads out the current pixels in the middle range to both ends of the spectrum. That is what histogram equalization can do.

Now, find the minimum histogram value (excluding 0) and apply a simple scaling of the CDF values back to the standard intensity range of [0, 255]. In theory, we normalize the CDF to [0, 1] before multiplying with the maximum of the range (L-1), which is 255. They will get us the same thing.

The concept of a masked array from Numpy is used here, that is, all operations are performed on non-masked elements. You can read more about it from Numpy docs on [masked arrays](https://docs.scipy.org/doc/numpy/reference/maskedarray.html).

In [ ]:
cdf_m = np.ma.masked_equal(cdf,0)
#print(cdf)
cdf_m = np.round((cdf_m - cdf_m.min())/(cdf_m.max()-cdf_m.min())*255)     # discretize back the CDF values to [0, 255]
#print(cdf_m)
new_cdf = np.ma.filled(cdf_m,0).astype('uint8')
#print(new_cdf)

# this does the mapping -- using the scaled cdf to look for the new intensity values
pout2 = new_cdf[pout]       

Use again the earlier code to draw the histogram and cdf plot together. Observe what happens to both the histogram and cdf plot. 

In [ ]:
hist2,bins = np.histogram(pout2.flatten(),256,[0,256])

cdf2 = hist2.cumsum()

cdf_normalized = cdf2 * hist2.max()/ cdf2.max()     # normalize just to scale values down to show it clearly within histogram
plt.plot(cdf_normalized, color = 'b')
plt.hist(pout2.flatten(),256,[0,256], color = 'r')
plt.xlim([0,256])
plt.legend(('cdf','histogram'), loc = 'upper left')
plt.show()
plt.imshow(pout2, cmap='gray'); plt.show()

OpenCV provides a function to perform histogram equalization, `equalizeHist`. 


In [ ]:
equalized = cv2.equalizeHist(pout)
pout_stack = np.hstack((pout,equalized)) #stacking images side-by-side. Useful function!
showImage(pout_stack)

**Q3**: What happens if you perform histogram equalization one more time?

In [ ]:
#Enter code here


**Q4. Histogram equalization on color images**. 

Perform histogram equalization on the following color image.

<img src="landscape.jpg" style="width:300px">

Display the original and equalized image along with the respective histograms.

In [ ]:
#Enter code here


## Additional Exercise



### Contrast Stretching

Contrast stretching is a way to improve the contrast in an image by "stretching" the range of intensity values it contains to span a desired range of values. Typical usage is to stretch the low-contrast range to the full range of pixel values (0 to 255 or whichever range the image type concerned allows).

It can be characterized by piecewise linear functions:
<img src="piecewise-linear.png" style="width:250px">
<br>
\begin{equation}y=\frac{b_{i+1}-b_{i}}{a_{i+1}-a_{i}}(x-a_{i})+b_{i}\end{equation}

You can see that the drastic transformation occurs to the middle section of the original range; the narrow $a_2-a_3$ range is stretched to a much wider $b_2-b_3$ range. Meanwhile both ends of the original range is compressed to a slightly narrower output range.You can try to apply non-linear piecewise contrast stretching to the images above and compare with results of other types of enhancement techniques.

**Q**. Write a function that that calculate the output pixel value based on a piecewise linear function specified by values $a_{1},a_{2},...$ and $b_{1},b_{2},...$. Test it out on the `pout.tif` image.

In [ ]:
#Enter code here


### Function Code Exercise

This exercise is to get you familiarized with the format required of the code submission in the upcoming individual assignment. In the provided materials, there is a folder containing a small dataset of 4 degraded images with the corresponding high quality groundtruths. Try to design a function to enhance these degraded images and place the code in the `adjustContrast.py` file. To evaluate the performance of your enhancement, just run the accompanying `evaluateContrast.py` which will compare the results with the groundtruth using 3 evaluation metrics, Mean Absolute Error (MAE), Peak Signal-to-Noise Ratio (PSNR), and Strutural Similarity (SSIM). <br>

In order for the codes to work, please pip install the `prettytable` package, and the evaluation results will be printed in your console.